In [3]:
import os
import argparse
import yaml
import glob
from tqdm import trange
import pandas as pd
import yaml
import torch
import scipy
import numpy as np
import argparse
import os
from glob import glob
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt

In [4]:
from dataset import CTDataset 

In [5]:
from train import create_dataloader, load_model 

In [ ]:
from dataset import CTDataset 
import os
import argparse
import yaml
import glob
from tqdm import trange
import pandas as pd
import yaml
import torch
import scipy
import numpy as np
import argparse
import os
from glob import glob
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
from train import create_dataloader, load_model 


In [9]:
## linux path: /home/azureuser/snow-Dayz/configs/exp_resnet50_2classes.yaml
### local path 
cfg = yaml.safe_load(open('//home/azureuser/snow-Dayz/configs/exp_resnet50_2classes.yaml'))
model = torch.load('/home/azureuser/snow-Dayz/experiments/experiment_name/model_states/5.pt')
#model1 = torch.load_state_dict('/home/azureuser/snow-Dayz/experiments/experiment_name/model_states/5.pt')
dataLoader= create_dataloader(cfg, split='train', labels = 'trainLabels.csv', folder = 'train')
exp_name = 'experiment_name'


FileNotFoundError: [Errno 2] No such file or directory: '/datadrive/vmData/weather/trainLabels.csv'

In [ ]:
model_instance = CustomResNet50(cfg['num_classes'])         # create an object instance of our CustomResNet18 class

    # load latest model state
model_states = glob(exp_name+'/model_states/*.pt')

if len(model_states) > 0:
        # at least one save state found; get latest
    model_epochs = [int(m.replace({exp_name}+'/model_states/','').replace('.pt','')) for m in model_states]
    if epoch:
        start_epoch = epoch
    else:
        start_epoch = max(model_epochs)

        # load state dict and apply weights to model
    print(f'Evaluating from epoch {start_epoch}')
    state = torch.load(open(f'{exp_name}/model_states/{start_epoch}.pt', 'rb'), map_location='cpu')
    model_instance.load_state_dict(state['model'])

        #import IPython
        #IPython.embed()

In [ ]:

def predict(cfg, dataLoader, model):
    with torch.no_grad(): # no gradients needed for prediction
        predictions = []
        predict_labels = [] 
        labels = []
        confidences = []

        model.eval()
        
        ##### may need to adjust this in the dataloader for the sequence:
        for idx, (data, label) in enumerate(dataLoader): 
            prediction = model(data) ## the full probabilty
            predict_label = torch.argmax(prediction, dim=1) ## the label
            confidence = torch.nn.Softmax(prediction)

        predictions.append(prediction)
        predict_labels.append(int(predict_label))
        labels.append(int(label))
        confidences.append(int(confidence))

    return predictions, predict_labels, labels, confidence


In [11]:
with torch.no_grad(): # no gradients needed for prediction
    predictions = []
    predict_labels = [] 
    labels = []
    confidences = []
        ##### may need to adjust this in the dataloader for the sequence:
    for idx, (data, label) in enumerate(dataLoader): 
        prediction = model(data) ## the full probabilty
        predict_label = torch.argmax(prediction, dim=1) ## the label
        confidence = torch.nn.Softmax(prediction)

    predictions.append(prediction)
    predict_labels.append(int(predict_label))
    labels.append(int(label))
    confidences.append(int(confidence))

True

In [17]:
def predict(dataLoader, model):
  with torch.no_grad():   # no gradients needed for prediction
    predictions = []
    predict_labels = []
    labels = []
    data = []

    model.eval()

    for idx, (data, label) in enumerate(dataLoader): 
      prediction = model(data) 
      predict_label = torch.argmax(prediction, dim=1) 

      predictions.append(prediction)
      predict_labels.append(int(predict_label))
      labels.append(int(label))

    return data, predictions, predict_labels, labels

In [16]:
def get_fuzzy_accuracy(y_true, y_pred):
    # OA: number of correct predictions divided by batch size (i.e., average/mean)
    facc = 0
    for true, pred in zip(y_true, y_pred):
        if pred in range(true - 1, true + 1, 1):
            facc += 1
    
    facc /= len(y_true)

    return facc

In [15]:
def save_confusion_matrix(y_true, y_pred, exp_name, epoch, split):
    # make figures folder if not there
    os.makedirs(exp_name+'/figs', exist_ok=True)

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(cm)
    disp.plot()
    plt.savefig(exp_name+'/figs/confusion_matrix_epoch'+str(epoch)+'_'+str(split)+'.png', facecolor="white")
    
    return cm


In [ ]:
# set model directory
#outdir = exp_name

    # get config from model directory
config = glob(exp_name+'*.yaml')[0]

    # load config
print(f'Using config "{config}" and using "{args.split}" set')
#cfg = yaml.safe_load(open(config, 'r'))

    # setup dataloader
dl_val = create_dataloader(cfg, split=args.split, batch=1)


In [18]:

    # load model and predict from model
#model, epoch = load_model(cfg, exp_name, args.epoch)
data, predictions, predict_labels, labels = predict(dl_val, model)   
    
    # get accuracy score
acc = accuracy_score(labels, predict_labels)
print("Accuracy of model is {:0.2f}".format(acc))
    
    # get fuzzy accuracy
facc = get_fuzzy_accuracy(labels, predict_labels)
print("Accuracy within 1 class is {:0.2f}".format(facc))

    # confusion matrix
cm = save_confusion_matrix(labels, predict_labels, exp_name, epoch, args.split)

    # save list of predictions with filename
df = pd.DataFrame({'filename': data,
                    #'predictions': predictions,
                    'predict_label': predict_labels,
                    'real_label': labels}
    )
df.to_csv(outdir+'/results_epoch'+str(epoch)+'_'+str(args.split)+'.csv', index = False)

    # precision recall curve


NameError: name 'outdir' is not defined